# ASE 从入门到精通 第一章

<div style="color:black; background-color:#FFF3E9; border: 1px solid #FFE0C3; border-radius: 10px; margin-bottom:0rem">
    <p style="margin:1rem; padding-left: 1rem; line-height: 2.5;">
        ©️ <b><i>Copyright 2023 @ Authors</i></b><br/>
        <i>作者：刘照清（波本）
            <b>
            <a href="mailto:quantummisaka@mail.pku.edu.cn">*** 📨 </a>
            </b>
        </i>
        <br/>
        <i>日期：2023-08-08</i><br/>
        <i>共享协议：</a>本作品采用<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">知识共享署名-非商业性使用-相同方式共享 4.0 国际许可协议</a>进行许可。</i><br/>
        <i>快速开始：点击上方的</i> <span style="background-color:rgb(85, 91, 228); color:white; padding: 3px; border-radius: 5px;box-shadow: 2px 2px 3px rgba(0, 0, 0, 0.3); font-size:0.75rem;">开始连接</span> <i>按钮，选择 <b><u>bohrium-notebook:2023-05-31 </u>镜像</b> 和任意配置机型即可开始。</i><br/>
        <i>第一章参考：</a><a rel="license" href="https://nb.bohrium.dp.tech/detail/8841808328#case"> 快速开始ASE | 氮气分子在Cu表面的吸附 </i>
    </p>
</div>

🎯 <b style='color:purple'>本教程旨在<u> 全面掌握</u> 使用 ASE 模块建立分子，晶体和表面模型并进行计算。</b>

* 从计算模拟的常用思路出发进行讲解，易于理解与上手

* 一键运行，你可以快速在实践中检验你的想法。

* 丰富完善的注释，对于入门者友好。

## 目标大纲

> **基于实例掌握 ASE 的建模，可视化与计算方法**

<div align="left" style="margin:1.5rem"><img src="https://gitlab.com/uploads/-/system/project/avatar/470007/ase256.png?width=64" alt="pandas" style="zoom: 200%;"></div>

1. 第一章：ASE快速入门
2. 第二章：ASE基础模块简析
3. 第三章：ASE表面吸附计算与数据库
4. 第四章：ASE全局优化方法
5. 第五章：ASE计算扩散与解离性质
6. 第六章：ASE分子动力学方法
7. 第七章：ASE外接其他软件
8. 第八章：ASE高级操作

## 第一章: ASE简介与快速入门 <a id='practice'></a>

本章将从$N_2$在Cu(111)表面的吸附为例来掌握ASE的建模，可视化与基本计算方法

## 本章目标

在学习本教程后，你将能够：

- 使用 ASE 定义分子或晶体。
- 计算体系能量并进行结构弛豫。
- 使用 ASE 读取或写入原子文件
- 可视化原子文件
- 使用 ASE 进行分子动力学计算

**阅读该教程【最多】约需 12 分钟，让我们开始吧！**

### 0：安装ASE

以Bohrium镜像为例检查ASE是否已经安装。
如果没有安装，我们采用pip快速安装

In [ ]:
# 以下命令判断该环境中是否存在所需模块，如果没有，则使用 pip 快速安装。
! if ! /opt/mamba/bin/pip show ase > /dev/null; then /opt/mamba/bin/pip install --upgrade --user ase; fi;

### 1 模型构建，可视化与SCF计算 <a id='case1'></a>

#### 1.1 原子对象 <a id='1-1'></a>

[Atoms](https://wiki.fysik.dtu.dk/ase/ase/atoms.html#ase.Atoms) 对象是原子的集合。以下是如何通过直接指定两个氮原子的位置来定义 $N_2$ 分子的方法

In [2]:
from ase import Atoms

d = 1.10
molecule = Atoms('2N', positions=[(0., 0., 0.), (0., 0., d)])

您也可以使用lattice模块构建晶体，该模块返回对应于常见晶体结构的 `Atoms` 对象。让我们制作一个Cu（111）表面：

In [3]:
from ase.build import fcc111

slab = fcc111('Cu', size=(4,4,2), vacuum=10.0)

直接通过代码构建模型的感觉怎么样？可能心里没有那种“所见即所得”的建模软件踏实。
让我们看看我们的模型构建的怎么样，这可以通过`ase.visualize`的`view()`方法完成

In [4]:
from ase.visualize import view

view(molecule)
view(slab) # can be done by vscode

<Popen: returncode: None args: ['/home/james/apps/anaconda3/envs/pyscf/bin/p...>

view() 将弹出一个 ase.gui 窗口，这个窗口是交互式的，可以更换可视化角度并选择原子等。

但请注意，默认的 viewer 使用的是 ase.gui 窗口，无法在 notebook 中展示。

可以通过指定可选的关键字 viewer=... 来使用备用查看器。

（请注意，这些备用查看器不是ASE的一部分，用户需要单独安装。） 

在 bohrium notebook 中，最佳的 viewer 参数是 `ngl` 或 `x3d` 

`ngl` 需要 nglview 的支持。一般来说，你的镜像已经安装了nglview，如果没有安装，请执行以下命令

#### 1.2 添加计算器 <a id='1-2'></a>

在这个ASE教程中，大部分的计算采用有效介质理论（EMT）计算器，因为它非常快速。
但需要注意的是，其他的计算器都可以通过`ase.calculator`附加在 ASE 模块上，
比如`VASP`, `GPAW`, `pyscf`, 以及`ABACUS`

我们可以将计算器附加到先前创建的 Atoms 对象上：

In [5]:
from ase.calculators.emt import EMT
slab.calc = EMT()
molecule.calc = EMT()

并使用Atoms类的get_potential_energy()方法来使用它计算系统的总能量：

In [6]:
e_slab = slab.get_potential_energy()
e_N2 = molecule.get_potential_energy()
print(f'N2 的能量是：{e_N2}')
print(f'Cu 的能量是：{e_slab}')

N2 的能量是：0.44034357303561467
Cu 的能量是：11.509056283570382


### 2 结构优化：以吸附态为例 <a id='case1'></a>

让我们使用拟牛顿优化（Quasi-Newton）方法优化 $N_2$ 分子在铜表面上吸附的结构。首先需要将吸附剂添加到铜片上，构建表面吸附模型

In [7]:
from ase.build import add_adsorbate

h = 1.85
add_adsorbate(slab, molecule, h, 'ontop')

为了更好地模拟“从体相延伸出去的表面”体系，也为了加快优化速度，一般来说计算表面吸附与反应体系时，会固定相对底面的1-2层原子。

简单起见，此处让我们使用来自 `ase.constraints` 约束模块的 `FixAtoms` 将所有 Cu 原子固定，只允许 $N_2$ 分子松弛到平衡结构。由于EMT计算器计算速度很快，这块原子固定操作可以不进行。

需要注意的是，被固定的原子仅仅是不参与驰豫，而不是不参与计算。因此，被固定的原子仍然会对体系的总能量产生影响。

In [8]:
from ase.constraints import FixAtoms

constraint = FixAtoms(mask=[a.symbol != 'N' for a in slab])
slab.set_constraint(constraint)

现在将拟牛顿最小化器连接到系统并保存轨迹文件。在给定的收敛准则运行优化计算，即所有原子的力应小于某个 fmax：

In [9]:
from ase.optimize import QuasiNewton

dyn = QuasiNewton(slab, trajectory='N2Cu.traj')
dyn.run(fmax=0.05)

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 23:03:22       11.689927*       1.0797
BFGSLineSearch:    1[  2] 23:03:23       11.670814*       0.4090
BFGSLineSearch:    2[  4] 23:03:23       11.625880*       0.0409


True

自然，这里也可以采用其他的优化器，比如优化中常用的`BFGS`, `CG`等等。

In [12]:
from ase.optimize import BFGS
import os
os.system("rm -f N2Cu.traj")

dyn2 = BFGS(slab, trajectory='N2Cu.traj')
dyn2.run(fmax=0.05)

      Step     Time          Energy         fmax
BFGS:    0 23:04:17       11.625880        0.0409


True

到这里我们就得到了弛豫后体系的总能量是 11.803869 eV。

让我们计算一下吸附能。

In [13]:
print('吸附能:', e_slab + e_N2 - slab.get_potential_energy())

吸附能: 0.3235194223180837


### 3 输入与输出 <a id='io'></a>

ASE的输入与输出主要通过`ase.io`模块中的方法进行

可以调用其中的 `write()` 函数将原子位置写入文件：

In [ ]:
from ase.io import write

write('slab.xyz', slab) # 保存为 extxyz 格式

这将以xyz格式编写文件。可能的格式有：

| 格式   | 描述                       |
| ------ | -------------------------- |
| `xyz`  | 常规的xyz格式（其实是`extxyz`格式）|
| `cube` | 高斯立方体文件             |
| `pdb`  | 蛋白质数据储存文件         |
| `traj` | ASE自己的轨迹格式          |
| `py`   | Python脚本                 |

此处的`extxyz`格式是`xyz`格式的加强版，既保留了`xyz`格式的基本信息，
又在此基础上增加了对体系各种属性（能量，原子受力，维里）等的读写支持，这种格式的文件被广泛应用于各种软件中，如分子动力学可视化软件Ovito，以及图神经网络机器学习势Nequip

从文件中读取化学信息则常用`ase.io`模块的`read()`函数，具体的方法如下：

In [ ]:
from ase.io import read

slab_from_file = read('slab.xyz')

### 4 案例总结：N2在Cu上的吸附 <a id='case1'></a>

将以上代码合并，即为通过ASE简单考察N2在Cu111表面吸附的全过程连续代码

In [14]:
from ase import Atoms
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.optimize import QuasiNewton
from ase.build import fcc111, add_adsorbate

h = 1.85
d = 1.10

slab = fcc111('Cu', size=(4, 4, 2), vacuum=10.0)

slab.calc = EMT()
e_slab = slab.get_potential_energy()

molecule = Atoms('2N', positions=[(0., 0., 0.), (0., 0., d)])
molecule.calc = EMT()
e_N2 = molecule.get_potential_energy()

add_adsorbate(slab, molecule, h, 'ontop')
constraint = FixAtoms(mask=[a.symbol != 'N' for a in slab])
slab.set_constraint(constraint)
dyn = QuasiNewton(slab, trajectory='N2Cu.traj')
dyn.run(fmax=0.05)

print('吸附能:', e_slab + e_N2 - slab.get_potential_energy())

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 23:19:19       11.689927*       1.0797
BFGSLineSearch:    1[  2] 23:19:19       11.670814*       0.4090
BFGSLineSearch:    2[  4] 23:19:19       11.625880*       0.0409
吸附能: 0.3235194223180837


同样可以使用`view()`函数对我们优化得到的表面吸附体系进行可视化

In [ ]:
from ase.visualize import view

view(slab) # can be done by vscode

### 5 分子动力学（Molecular dynamics）计算示例 <a id='molecular-dynamics'></a>

ase中的`ase.md`模块可以进行分子动力学计算，这里让我们以氮分子为例，用 VelocityVerlet 算法来简单运行分子动力学模拟。

我们首先创建 VelocityVerlet 对象，将其赋予分子（即N2的Atoms对象）和用于牛顿定律积分的时间步长。

然后，我们通过调用其 run() 方法并告诉它要执行多少步骤来执行动力学：

In [17]:
from ase.md.verlet import VelocityVerlet
from ase import units # units模块用于定义单位

dyn = VelocityVerlet(molecule, dt=1.0 * units.fs) 

for i in range(10):
    pot = molecule.get_potential_energy()
    kin = molecule.get_kinetic_energy()
    print('%2d: %.5f eV, %.5f eV, %.5f eV' % (i, pot + kin, pot, kin))
    dyn.run(steps=50)

 0: 0.43838 eV, 0.29518 eV, 0.14320 eV
 1: 0.43909 eV, 0.31878 eV, 0.12031 eV
 2: 0.44034 eV, 0.44008 eV, 0.00026 eV
 3: 0.43926 eV, 0.33273 eV, 0.10653 eV
 4: 0.43824 eV, 0.28343 eV, 0.15482 eV
 5: 0.44073 eV, 0.43975 eV, 0.00098 eV
 6: 0.43846 eV, 0.30187 eV, 0.13659 eV
 7: 0.43900 eV, 0.31207 eV, 0.12693 eV
 8: 0.44033 eV, 0.43928 eV, 0.00106 eV
 9: 0.43934 eV, 0.33987 eV, 0.09947 eV


### 6 本章总结 <a id='summary'></a>

In [ ]:
# 运行完毕后删除生成的临时文件
import os

tmp_files = ['N2Cu.traj', 'slab.xyz']
for i in tmp_files:
    os.remove(i)

在本教程中，您学习了在 ASE 中的一些基础方法。 

具体而言，您了解到： 
- 使用 ASE 定义分子或晶体。
- 计算体系能量并进行结构弛豫。
- 使用 ASE 读取或写入原子文件
- 可视化原子文件
- 使用 ASE 进行分子动力学计算
 
你有什么问题吗？ 欢迎与我们联系 [bohrium@dp.tech](mailto:bohrium@dp.tech) 。